# Chunking

This code was taken from Saric (https://github.com/SanjaSaric/HSA-Topics/blob/master/Chunking.ipynb)

In [19]:
from pathlib import Path
import os 
import re

In [20]:
data = 'C:/Users/elisa/DH-MA/data' 

## Laden und sortieren

In [21]:
path_to_corpus = Path(data, 'fwp') # Ausgangspfad (wird nicht überschrieben)

In [22]:
sorted(os.listdir(path=path_to_corpus))

['alabama_1.txt',
 'arkansas_1.txt',
 'arkansas_2.txt',
 'arkansas_3.txt',
 'arkansas_4.txt',
 'arkansas_5.txt',
 'arkansas_6.txt',
 'arkansas_7.txt',
 'florida_1.txt',
 'georgia_1.txt',
 'georgia_2.txt',
 'georgia_3.txt',
 'georgia_4.txt',
 'indiana_1.txt',
 'kansas_1.txt',
 'kentucky_1.txt',
 'maryland_1.txt',
 'mississippi_1.txt',
 'missouri_1.txt',
 'northcarolina_1.txt',
 'northcarolina_2.txt',
 'ohio_1.txt',
 'oklahoma_1.txt',
 'southcarolina_1.txt',
 'southcarolina_2.txt',
 'southcarolina_3.txt',
 'southcarolina_4.txt',
 'tennessee_1.txt',
 'texas_1.txt',
 'texas_2.txt',
 'texas_3.txt',
 'texas_4.txt',
 'virginia_1.txt']

In [23]:
filenames = [os.path.join(path_to_corpus, fn) for fn in sorted(os.listdir(path_to_corpus))]
filenames

['C:\\Users\\elisa\\DH-MA\\data\\fwp\\alabama_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_2.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_3.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_4.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_5.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_6.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\arkansas_7.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\florida_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\georgia_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\georgia_2.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\georgia_3.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\georgia_4.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\indiana_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\kansas_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\kentucky_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\maryland_1.txt',
 'C:\\Users\\elisa\\DH-MA\\data\\fwp\\mississippi_1.txt',
 'C:\\Users\\eli

## Dokumente in chunks teilen

In [24]:
def split_text(filename, n_words):
    """Split a text into chunks approximately `n_words` words in length."""
    input = open(filename, 'r', encoding="utf-8")
    words = " ".join(re.sub(',|\.|\;|\:|\(|\)|\-','',input.read()).split()).split(' ') # remove special charachters and normalize space
    input.close()
    chunks = []
    current_chunk_words = []
    current_chunk_word_count = 0
    for word in words:
        current_chunk_words.append(word)
        current_chunk_word_count += 1
        if current_chunk_word_count == n_words:
            chunks.append(' '.join(current_chunk_words))
            current_chunk_words = []
            current_chunk_word_count = 0
    chunks.append(' '.join(current_chunk_words) )
    return chunks

In [25]:
filenames.sort()

In [26]:
chunk_length = 10000
chunks = []

for filename in filenames:
    chunk_counter = 0
    texts = split_text(filename, chunk_length)
    for text in texts:
        chunk = {'text': text, 'number': chunk_counter, 'filename': filename} # make dictionary with content and information
        chunks.append(chunk)
        chunk_counter += 1
        

Anzahl der Originaldateien:

In [27]:
len(filenames)

33

Anzahl der erzeugten chunks:

In [28]:
len(chunks)

272

Prüfen, ob eine Originaldatei zu kurz war, um sie im Topic Modeling überhaupt zu verwenden. 

In [11]:
i = 0
for chunk in chunks:
    l_chunk = len(chunk['text'].split(' '))
    if l_chunk < 3000 and chunk['number'] == 0:
        i+=1
        print(l_chunk, chunk['filename'],chunk['number'])
print('Anzahl der Dateien, die du entfernen solltest: ', i)

Anzahl der Dateien, die du entfernen solltest:  0


Hatte eine Datei beispielsweise 110 Tokens, werden 2 chunks produziert: <br>
1) mit 100 Tokens <br>
2) mit 10 Tokens. <br>
Wir möchten diese kurzen chunks ihren vorhergehenden Geschwisterdateien hinzufügen.  

In [12]:
i = 0
for chunk in chunks:
    index = chunks.index(chunk)
    l_chunk = len(chunk['text'].split(' '))
    if l_chunk < 3000 and chunk['number'] != 0:
        i+=1
        chunks[index-1]['text'] = chunks[index-1]['text'] + ' ' + chunk['text']
        print('Chunk ' + chunk['filename'] + str(chunk['number']-1) + ' erweitert mit chunk ' + str(chunk['number']) + ' auf dem Index ' + str(index))
        
print('Anzahl der erweiterten chunks: ' + str(i))

Chunk C:\Users\elisa\DH-MA\data\fwp\arkansas_1.txt8 erweitert mit chunk 9 auf dem Index 22
Chunk C:\Users\elisa\DH-MA\data\fwp\arkansas_4.txt7 erweitert mit chunk 8 auf dem Index 52
Chunk C:\Users\elisa\DH-MA\data\fwp\florida_1.txt7 erweitert mit chunk 8 auf dem Index 88
Chunk C:\Users\elisa\DH-MA\data\fwp\georgia_2.txt8 erweitert mit chunk 9 auf dem Index 108
Chunk C:\Users\elisa\DH-MA\data\fwp\kentucky_1.txt3 erweitert mit chunk 4 auf dem Index 140
Chunk C:\Users\elisa\DH-MA\data\fwp\maryland_1.txt1 erweitert mit chunk 2 auf dem Index 143
Chunk C:\Users\elisa\DH-MA\data\fwp\missouri_1.txt9 erweitert mit chunk 10 auf dem Index 159
Chunk C:\Users\elisa\DH-MA\data\fwp\northcarolina_1.txt8 erweitert mit chunk 9 auf dem Index 169
Chunk C:\Users\elisa\DH-MA\data\fwp\northcarolina_2.txt7 erweitert mit chunk 8 auf dem Index 178
Chunk C:\Users\elisa\DH-MA\data\fwp\southcarolina_3.txt7 erweitert mit chunk 8 auf dem Index 223
Chunk C:\Users\elisa\DH-MA\data\fwp\tennessee_1.txt1 erweitert mit ch

In [13]:
#chunks

Nun können diejenigen chunks, die bereits zu ihren Geschwisterdateien kopiert wurden, sowie diejenigen chunks, die sehr kurz waren und keine Geschwister hatten (= kurze Originalfiles) gelöscht werden.

In [14]:
i = 0
for chunk in chunks:
    index = chunks.index(chunk)
    l_chunk = len(chunk['text'].split(' '))
    if l_chunk < 3000:
        i+=1
        chunks.remove(chunk)
        
print('Gelöschte chunks: ' + str(i))

Gelöschte chunks: 13


In [15]:
print('Übriggebliebene: ' + str(len(chunks)))

Übriggebliebene: 259


## chunks zu Textdateien speichern

In [16]:
output_dir = 'C:/Users/elisa/DH-MA/data/fwp-chunks'

In [17]:
""" Quelle für Code: DARIAH-DE (https://liferay.de.dariah.eu/tatom/index.html)

    for chunk in chunks:
    basename = os.path.basename(chunk['filename'])
    fn = os.path.join(output_dir, "{}{:04d}".format(basename, chunk['number']))
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(chunk['text'])
"""
# umgeändert, sodass valide txt-Dateien als output kommen
for chunk in chunks:
    basename = os.path.basename(chunk['filename'])
    fn_base, fn_ext = os.path.splitext(basename)
    fn = os.path.join(output_dir, "{}_{:04d}{}".format(fn_base, chunk['number'], fn_ext))
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(chunk['text'])

Testen, ob kurze Dateien übriggeblieben sind:

In [18]:
# Test if short files remained
i = 0
for chunkfile in Path(data, output_dir).glob('*.txt'):
    with open(chunkfile, encoding='utf-8') as f:
        text = f.read().split(' ')
        #print(len(text))
        if len(text) < 3000:
            i+=1
            print(chunkfile.name)
print('Übriggebliebene kurze Files: ', i)

Übriggebliebene kurze Files:  0


Wenn ja, dann sollten diese auch manuell entfernt oder zu Geschwisterdateien hinzugefügt werden.
